## Farm/farm.py

### Imports & settings

import the base (farm, field, ...) and the entities you want in your farm.

In [5]:
from farmgym.v2.farm import Farm
from farmgym.v2.field import Field
from farmgym.v2.farmers.BasicFarmer import BasicFarmer
from farmgym.v2.scorings.BasicScore import BasicScore
from farmgym.v2.rules.BasicRule import BasicRule
from farmgym.v2.entities.Weather import Weather
from farmgym.v2.entities.Soil import Soil
from farmgym.v2.entities.Plant import Plant

from farmgym.v2.entities.Birds import Birds
from farmgym.v2.entities.Facilities import Facility
from farmgym.v2.entities.Fertilizer import Fertilizer
from farmgym.v2.entities.Weeds import Weeds
from farmgym.v2.entities.Pests import Pests
from farmgym.v2.entities.Pollinators import Pollinators
from farmgym.v2.entities.Cide import Cide


from farmgym.v2.rendering.monitoring import mat2d_value, sum_value


from gym.envs.registration import register

import os
from pathlib import Path

file_path = Path(os.path.realpath(__file__))
CURRENT_DIR = file_path.parent


NameError: name '__file__' is not defined

### env() function

For more details on the existing settings, you can check documentation on https://gitlab.inria.fr/rl4ae/farm-gym .

 - 1 : you have to create your field, with available (and mendatory) entities for you farm. (check documentation for the options)
 - 2 : Set the rules of your farm. 'free observation' that will be use for the learning , terminal condition, and specific configurations
 - 3 : create you farm with the previous settings
 - 4 : add the "unobservable farm values" to the farm monitoring
 - 5 : return the farm
 
 

In [6]:


def env():
    ##########################################################################
    entities1 = []
    entities1.append((Weather, "lille"))
    entities1.append((Soil, "clay"))
    entities1.append((Plant, "bean"))

    entities1.append((Fertilizer, "basic_all"))

    entities1.append((Pollinators, "bee"))

    entities1.append((Weeds, "base_weed"))
    entities1.append((Cide, "herbicide"))

    entities1.append((Pests, "basic"))
    entities1.append((Pollinators, "bee"))
    entities1.append((Cide, "pesticide"))

    field1 = Field(
        localization={"latitude#°": 50.38, "longitude#°": 3.03, "altitude#m": 10},
        shape={"length#nb": 1, "width#nb": 1, "scale#m": 1.0},
        entity_managers=entities1,
    )

    farmer1 = BasicFarmer(max_daily_interventions=1)
    scoring = BasicScore(score_configuration=CURRENT_DIR / "farm_score.yaml")

    free_observations = []
    free_observations.append(("Field-0", "Weather-0", "day#int365", []))
    free_observations.append(("Field-0", "Weather-0", "air_temperature", []))
    free_observations.append(("Field-0", "Weather-0", "rain_amount", []))
    free_observations.append(("Field-0", "Weather-0", "sun_exposure#int5", []))
    free_observations.append(("Field-0", "Weather-0", "consecutive_dry#day", []))

    free_observations.append(("Field-0", "Plant-0", "stage", []))
    free_observations.append(("Field-0", "Plant-0", "fruits_per_plant#nb", []))

    free_observations.append(("Field-0", "Plant-0", "size#cm", []))

    free_observations.append(("Field-0", "Soil-0", "wet_surface#m2.day-1", []))

    free_observations.append(("Field-0", "Fertilizer-0", "amount#kg", []))
    free_observations.append(("Field-0", "Pollinators-0", "occurrence#bin", []))
    free_observations.append(("Field-0", "Weeds-0", "grow#nb", []))
    free_observations.append(("Field-0", "Weeds-0", "flowers#nb", []))
    free_observations.append(("Field-0", "Plant-0", "fruit_weight#g", []))

    free_observations.append(("Field-0", "Soil-0", "microlife_health_index#%", []))

    terminal_CNF_conditions = [
        [(("Field-0", "Weather-0", "day#int365", []), lambda x: x.value, ">=", 360)],
        [(("Field-0", "Plant-0", "global_stage", []), lambda x: x.value, "==", "dead")],
    ]
    rules = BasicRule(
        init_configuration=CURRENT_DIR / "farm_init.yaml",
        actions_configuration=CURRENT_DIR / "farm_actions.yaml",
        terminal_CNF_conditions=terminal_CNF_conditions,
        free_observations=free_observations,
    )

    farm = Farm(fields=[field1], farmers=[farmer1], scoring=scoring, rules=rules)

    ##########################################################################

    var = []

    var.append(
        (
            "Field-0",
            "Soil-0",
            "available_N#g",
            lambda x: sum_value(x),
            "Available Nitrogen (g)",
            "range_auto",
        )
    )
    var.append(
        (
            "Field-0",
            "Soil-0",
            "available_Water#L",
            lambda x: sum_value(x),
            "Available Water (g)",
            "range_auto",
        )
    )
    var.append(
        (
            "Field-0",
            "Plant-0",
            "size#cm",
            lambda x: sum_value(x),
            "Size (cm)",
            "range_auto",
        )
    )
    var.append(
        (
            "Field-0",
            "Plant-0",
            "flowers_per_plant#nb",
            lambda x: sum_value(x),
            "Flowers (nb)",
            "range_auto",
        )
    )
    var.append(
        (
            "Field-0",
            "Plant-0",
            "flowers_pollinated_per_plant#nb",
            lambda x: sum_value(x),
            "Flowers pollinated (nb)",
            "range_auto",
        )
    )

    var.append(
        (
            "Field-0",
            "Plant-0",
            "fruit_weight#g",
            lambda x: sum_value(x),
            "Fruits weight (g)",
            "range_auto",
        )
    )

    var.append(
        (
            "Field-0",
            "Pests-0",
            "plot_population#nb",
            lambda x: x[(0, 0)].value,
            "Insects (nb)",
            "range_auto",
        )
    )
    var.append(
        (
            "Field-0",
            "Soil-0",
            "amount_cide#g",
            lambda x: x["soil"][0, 0].value,
            "Herbicide + pesticide in soil",
            "range_auto",
        )
    )
    var.append(
        (
            "Field-0",
            "Weeds-0",
            "seeds#nb",
            lambda x: x[0, 0].value,
            "Weeds seed nb",
            "range_auto",
        )
    )

    farm.add_monitoring(var)
    farm.monitor_variables = var

    return farm



## game2_env.py

### Imports

In [1]:
import rlberry.spaces as spaces
from rlberry.envs.interface import Model
import rlberry_farms.farm1.farm as cb
import numpy as np
import time
import os
from rlberry.utils.writers import DefaultWriter
from rlberry_farms.utils import (
    farmgymobs_to_obs,
    update_farm_writer,
    observation_hide_final_state_of_plants,
)


### Class Farm

"Farm" is the environment for the RL agent. It need reset and step functions.

Farm2 is a tutorial 1x1 farm with only one possible plant : beans, planted in a clay ground.
The advised maximum episode length is 365 (as in 365 days).

The Farm has the weather of Lille in France (e.g. well suited for the culture of beans), the initial day is 120. Initially the field is healthy and contains all the nutrient necessary to the plant.

The reward is the number of grams of harvested beans, and there is a negative reward for very low microlife in soil (due to pesticides).

the final state of plant (when it's ready to harvest) is hide, to avoid too easy observation.

The condition for ending the episode (self.step returns done) is that the day is >= 365 or that the plant is dead.

##### Parameters
----------
    monitor: boolean, default = True
        If monitor is True, then some (unobserved) variables are saved to a writer that is displayed during training.
    enable_tensorboard: boolean, default = False
        If True and monitor is True, save writer as tensorboard data
    output_dir: str, default = "results"
        directory where writer data are saved

##### Notes
----------
State:
The state consists of
 - Day (from 1 to 365)
 - mean air temperature (°C)
 - min air temperature (°C)
 - max air temperature (°C)
 - rain amount (mm)
 - sun-exposure (from 1 to 5)
 - consecutive dry day (int)
 - stage of growth of the plant (int)
 - size of the plant in cm.
 - Soil wet_surface#m2.day-1
 - fertilizer amount#kg
 - Pests plot_population#nb
 - Pollinators occurrence#bin
 - Weeds grow#nb
 - Weeds flowers#nb
 - Weight of fruits (g)
 - Microlife health index (%)

Actions:
The actions are :
 - doing nothing.
 - 2 levels of watering the field (1L or 5L of water)
 - harvesting
 - sow some seeds
 - scatter fertilizer
 - scatter herbicide
 - scatter pesticide
 - remove weeds by hand
 
 
##### Comments on some functions
----------

**init :** 
implement rlberry env interface, and use the previous 'farm' class
high and low are array for the min and max values for the observations

**step :**
farmgym run with a cycle of 2 steps: 1 (empty) step of getting observation ("morning"), then 1 step of acting ("afternoon").
Classic RL methodology use only 1 step : performing an action return the next observation
To match this 2, rlberry_farms run the 'farmgy observation step ("morning")' right after the action.
With this method, it will be like classic RL 'step' for the user

**num_to_action :**
convert the num of the action given by the agent, to the real action (intervention) to do.

In [2]:
class Farm2(Model):

    name = "Farm1"

    observations_txt = [
        "Day (from 1 to 365)",
        "Mean air temperature (°C)",
        "Min air temperature (°C)",
        "Max air temperature (°C)",
        "Rain amount",
        "Sun-exposure (from 1 to 5)",
        "Consecutive dry day (int)",
        "Stage of growth of the plant",
        "Number of fruits (int)",
        "Size of the plant in cm",
        "Soil wet_surface (m2.day-1)",
        "fertilizer amount (kg)",
        "Pollinators occurrence (bin)",
        "Weeds grow (nb)",
        "Weeds flowers (nb)",
        "weight of fruits",
        "microlife health index (%)",
    ]

    def __init__(self, monitor=False, enable_tensorboard=False, output_dir="results"):
        # init base classes
        Model.__init__(self)

        self.farm = cb.env()
        self.farm.gym_step([])

        self.farm.monitor = None
        # observation and action spaces
        # Day, temp mean, temp min, temp max, rain amount, sun exposure, consecutive dry day, stage, size#cm, nb of fruits,
        # wet surface,  fertilizer amount,  pollinators occurrence, weeds grow nb, weeds flower nb, weight of fruits, microlife health index %
        high = np.array(
            [365, 50, 50, 50, 300, 10, 10, 10, 100, 300, 10, 10, 1, 100, 100, 5000, 100]
        )
        low = np.array([0, -50, -50, -50, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
        self.observation_space = spaces.Box(low=low, high=high)
        self.action_space = spaces.Discrete(9)

        # monitoring writer
        self.identifier = self.name + str(self.seeder.rng.integers(100000))
        params = {}
        self.output_dir = output_dir
        if enable_tensorboard:
            self.tensorboard_dir = os.path.join(output_dir, "tensorboard")
            params["tensorboard_kwargs"] = dict(
                log_dir=os.path.join(self.tensorboard_dir, "farm_" + self.identifier)
            )
        self.writer = DefaultWriter(name="farm_writer", print_log=False, **params)
        self.monitor_variables = self.farm.monitor_variables
        self.iteration = 0
        self.monitor = monitor

        # initialize
        self.state = None
        self.reset()

    def reset(self):
        observation = self.farm.gym_reset()
        self.farm.gym_step([])
        return observation_hide_final_state_of_plants(
            farmgymobs_to_obs(observation), id_of_plants_stage=7
        )

    def writer_to_csv(self):
        self.writer.data.to_csv(
            os.path.join(self.output_dir, "farm_" + self.identifier + "_writer.csv")
        )

    def step(self, action):
        # Stepping
        #   farmgym run with a cycle of 2 steps: 1 (empty) step of getting observation ("morning"), then 1 step of acting ("afternoon").
        #   Classic RL methodology use only 1 step : performing an action return the next observation
        #   To match this 2, rlberry_farms run the 'farmgy observation step ("morning")' right after the action.
        #   With this method, it will be like classic RL 'step' for the user
        _, reward, is_done, info = self.farm.farmgym_step(self.num_to_action(action))
        obs1, _, _, info = self.farm.gym_step([])

        if hasattr(reward, "__len__"):
            reward = reward[0]

        # Monitoring
        if self.monitor:
            self.iteration += 1
            update_farm_writer(
                self.writer, self.monitor_variables, self.farm, self.iteration
            )
        if np.array(obs1[-1]).item() < 10:
            reward -= 2  # if microlife is < 10%, negative reward

        observation = observation_hide_final_state_of_plants(
            farmgymobs_to_obs(obs1), id_of_plants_stage=7
        )
        return observation, reward, is_done, info

    def num_to_action(self, num):
        if num == 1:
            return [
                (
                    "BasicFarmer-0",
                    "Field-0",
                    "Soil-0",
                    "water_discrete",
                    {"plot": (0, 0), "amount#L": 1, "duration#min": 60},
                )
            ]
        elif num == 2:
            return [
                (
                    "BasicFarmer-0",
                    "Field-0",
                    "Soil-0",
                    "water_discrete",
                    {"plot": (0, 0), "amount#L": 5, "duration#min": 60},
                )
            ]
        elif num == 3:
            return [("BasicFarmer-0", "Field-0", "Plant-0", "harvest", {})]
        elif num == 4:
            return [
                (
                    "BasicFarmer-0",
                    "Field-0",
                    "Plant-0",
                    "sow",
                    {"plot": (0, 0), "amount#seed": 1, "spacing#cm": 10},
                )
            ]
        elif num == 5:
            return [
                (
                    "BasicFarmer-0",
                    "Field-0",
                    "Fertilizer-0",
                    "scatter_bag",
                    {"plot": (0, 0), "amount#bag": 1},
                )
            ]
        elif num == 6:
            return [
                (
                    "BasicFarmer-0",
                    "Field-0",
                    "Cide-0",
                    "scatter_bag",
                    {"plot": (0, 0), "amount#bag": 1},
                )
            ]
        elif num == 7:
            return [
                (
                    "BasicFarmer-0",
                    "Field-0",
                    "Cide-1",
                    "scatter_bag",
                    {"plot": (0, 0), "amount#bag": 1},
                )
            ]
        elif num == 8:
            return [("BasicFarmer-0", "Field-0", "Weeds-0", "remove", {"plot": (0, 0)})]
        else:
            return []  # Do nothing.
